<a href="https://colab.research.google.com/github/afortuny/DeepLearningProduction/blob/main/Image_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide 
!pip install -Uqq fastbook # remember to restart runtime every time you run this line
import fastbook
fastbook.setup_book()

In [3]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [5]:
! pip install simple_image_download

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done


In [6]:
from simple_image_download import simple_image_download as simp



In [7]:
from simple_image_download import simple_image_download as simp 
response = simp.simple_image_download

In [8]:
lst=["adidas running shoes"]

In [9]:
for rep in lst:
 response().download(rep , 100)

In [10]:
dls = ImageDataLoaders.from_folder(Path('/content/simple_images'),valid_pct=0.2)

In [11]:
dls.valid_ds.items[:3]

[Path('/content/simple_images/adidas running shoes/adidas running shoes_22.jpg'),
 Path('/content/simple_images/adidas running shoes/adidas running shoes_16.jpg'),
 Path('/content/simple_images/adidas running shoes/adidas running shoes_27.jpg')]

In [12]:
def get_dls(size, bs, workers=None):
    source = Path('/content/simple_images')
    
    def dummy_label(o): return 0
    files = get_image_files(source)
    tfms = [[PILImage.create, ToTensor, RandomResizedCrop(size, min_scale=0.9)], [dummy_label]]
    
    dsets = Datasets(files, tfms=tfms, splits=RandomSplitter(valid_pct=0.1)(files))
    
    batch_tfms = [IntToFloatTensor]
    dls = dsets.dataloaders(bs=bs, num_workers=workers, after_batch=batch_tfms, device='cpu')
    return dls

In [13]:
bs,resize,size = 64,160,128
dls = get_dls(resize, bs)

In [ ]:
!pip install self-supervised

In [15]:
from self_supervised.layers import *

In [16]:
from self_supervised.vision.moco import *

In [17]:
encoder = create_encoder("tf_efficientnet_b4_ns", n_in=3, pretrained=False) # a timm encoder
model = create_moco_model(encoder, hidden_size=2048, projection_size=128)
aug_pipelines = get_moco_aug_pipelines(size=size)
learn = Learner(dls, model,loss_func=noop,cbs=[MOCO(aug_pipelines=aug_pipelines, K=128)])
learn.fit_flat_cos(100, 1e-2)

epoch,train_loss,valid_loss,time


/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


UnidentifiedImageError: ignored